In [14]:
from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds
from pprint import pprint

In [15]:
import time
import pandas as pd
from io import StringIO

In [16]:
import os
import time
import firebase_admin
from firebase_admin import credentials, firestore
import datetime

In [17]:
def initialize_firebase():
    cred_path = "treehacks-c0d12-firebase-adminsdk-fbsvc-02119ca686.json"
    if not firebase_admin._apps:  
        cred = credentials.Certificate(cred_path)
        firebase_admin.initialize_app(cred)

In [18]:
initialize_firebase()
db = firestore.client()

In [22]:
params = BrainFlowInputParams()
params.serial_port = "/dev/cu.Bluetooth-Incoming-Port"
board_id = BoardIds.CYTON_BOARD
board = BoardShim(board_id, params) 

In [23]:
eeg_channel_indices = BoardShim.get_eeg_channels(board_id)

In [ ]:
board_desc = BoardShim.get_board_descr(board_id)
pprint(board_desc)

In [ ]:
board.prepare_session()

In [ ]:
while (True):
    start_time = datetime.datetime.now()
    board.start_stream(10)
    time.sleep(10)
    data = board.get_board_data()  
    board.stop_stream()
    eeg_data = data[eeg_channel_indices, :]
    df = pd.DataFrame(eeg_data)
    csv_string = df.to_csv(index=False)
    doc_ref = db.collection("stream").add({
        "brain": csv_string,
        "timestamp": firestore.SERVER_TIMESTAMP  
    })
    print(f"Saved command to Firestore: {doc_ref[1].id}")
    time.sleep(5)

In [13]:
board.release_session()

In [12]:
def get_fine_tuning_data(num_samples):
    for i in range(num_samples):
        board.start_stream()
        time.sleep(5)
        data = board.get_board_data() 
        board.stop_stream()
        board.release_session()
        eeg_data = data[eeg_channel_indices, :]
        df = pd.DataFrame(eeg_data)
        df.to_csv(f"eeg-fine-tuning-data-{i}.csv", index=False)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

sampling_rate = 250 #hz
n_samples = df.shape[1]
time_vector = np.arange(n_samples) / sampling_rate 

fig, axes = plt.subplots(16, 1, figsize=(15, 30), sharex=True)

for i, channel in enumerate(df.index):
    axes[i].plot(time_vector, df.loc[channel], label=channel)
    axes[i].set_ylabel('Amplitude (uV)')
    axes[i].legend(loc='upper right')
    
axes[-1].set_xlabel('Time (s)')

plt.tight_layout()
plt.show()